# Integration Tests for Deno Jupyter
This notebook contains a number of tests to ensure that Jupyter is working as expected. You should be able to select "Kernel -> Restart Kernel and Run All" in Jupyter's notebook UI to run the tests.

## Passing Tests

### Simple Tests

#### This test should print "hi".
If this doesn't work, everything else will probably fail :)

In [1]:
console.log("hi")

hi


#### Top-level await

In [2]:
let x = await Promise.resolve(42);
console.log("x is", x);

x is 42


#### TypeScript transpiling
Credit to [typescriptlang.org](https://www.typescriptlang.org/docs/handbook/interfaces.html) for this code

In [3]:
interface SquareConfig {
  color?: string;
  width?: number;
}
 
function createSquare(config: SquareConfig): { color: string; area: number } {
  return {
    color: config.color || "red",
    area: config.width ? config.width * config.width : 20,
  };
}
 
createSquare({ colour: "red", width: 100 });

{ color: "red", area: 10000 }

### Return Values

#### undefined should not return a value

In [4]:
undefined

#### null should return "null"

In [5]:
null

null

#### boolean should return the boolean

In [6]:
true

true

#### number should return the number

In [7]:
42

42

#### string should return the string

In [8]:
"this is a test of the emergency broadcast system"

"this is a test of the emergency broadcast system"

#### bigint should return the bigint in literal format

In [9]:
31337n

31337n

#### symbol should return a string describing the symbol

In [12]:
Symbol("foo")

Symbol(foo)

#### object should describe the object inspection

In [14]:
{foo: "bar"}

{ foo: "bar" }

#### resolve returned promise

In [15]:
Promise.resolve("it worked!")

Promise { "it worked!" }

In [16]:
Promise.reject(new Error("it failed!"));

Promise {
  <rejected> Error: it failed!
    at <anonymous>:2:16
}

In [17]:
(function foo() {
    throw new Error("this is a test")
})()

Error: this is a test

In [18]:
throw "a party"

Unknown error: a party

In [20]:
Deno.readFile(1)

Promise { <pending> }

In [21]:
console.log("Hello from Deno!")

Hello from Deno!


In [22]:
console.log("%c Hello Deno ", "background-color: #15803d; color: white;");

 Hello Deno 


In [23]:
"Cool 🫡"

"Cool 🫡"

In [24]:
console.table([1, 2, 3])

┌───────┬────────┐
│ (idx) │ Values │
├───────┼────────┤
│     0 │      1 │
│     1 │      2 │
│     2 │      3 │
└───────┴────────┘


## Unit Tests With `Deno.test()`

In [25]:
Deno.test("passing test", () => {});

Deno.test("passing test with steps", async (t) => {
  await t.step("step 1", () => {});
  await t.step("step 2", () => {});
});

Deno.test("failing test", () => {
  throw new Error("some message");
});


passing test ... ok (1ms)
passing test with steps ...
  step 1 ... ok (1ms)
  step 2 ... ok (0ms)
passing test with steps ... ok (1ms)
failing test ... FAILED (2ms)

 ERRORS 

failing test => <anonymous>:7:6
error: Error: some message
    at <anonymous>:8:9

 FAILURES 

failing test => <anonymous>:7:6

FAILED | 2 passed (2 steps) | 1 failed (0ms)


## Broadcasting Display Updates

In [26]:
await Deno.jupyter.broadcast("display_data", {
    data: { "text/html": "<b>Processing.</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

await new Promise((resolve) => setTimeout(resolve, 500));

await Deno.jupyter.broadcast("update_display_data", {
    data: { "text/html": "<b>Processing..</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

await new Promise((resolve) => setTimeout(resolve, 500));

await Deno.jupyter.broadcast("update_display_data", {
    data: { "text/html": "<b>Processing...</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

await new Promise((resolve) => setTimeout(resolve, 500));

await Deno.jupyter.broadcast("update_display_data", {
    data: { "text/html": "<b>Complete ✅</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

Complete ✅

## Comms

In [27]:
// Smoke test: Send example Jupyter Widgets messages with "extra" context.
// No return because we don't have a front-end widget to get the message from.
await Deno.jupyter.broadcast(
    "comm_open",
    {
        "comm_id": "foo",
        "target_name": "jupyter.widget",
        "data": {
            "state": {},
        },
    },
    {
        "metadata": { "version": "2.1.0" },
    },
);

await Deno.jupyter.broadcast(
    "comm_msg",
    {
        "comm_id": "foo",
        "data": {
            "method": "update",
            "state": { "answer": null },
            "buffer_paths": [["answer"]]
        },
    },
    {
        "buffers": [new Uint8Array([42])],
    },
);

## Rich Classes

In [33]:
class SuperColor {
  constructor() {
      this.color = "#ff5398"
  }
  hex() {
      return this.color
  }
    
  [Symbol.for("Jupyter.display")]() {
      return {
          "text/html": `<div style="width: 32px; height: 32px; background-color: ${this.hex()}" />`
      }
  }
}

let sc = new SuperColor()
sc

In [32]:
class SuperColorAsync {
  constructor() {
      this.color = "#5398ff"
  }
  hex() {
      return this.color
  }
    
  async [Symbol.for("Jupyter.display")]() {
      return {
          "text/html": `<div style="width: 32px; height: 32px; background-color: ${this.hex()}" />`
      }
  }
}

let sc = new SuperColorAsync()
sc

## Some classes will have built in support / handling via `Deno.jupyter.format` under the hood.

In [36]:
import pl from "npm:nodejs-polars";

let response = await fetch(
  "https://cdn.jsdelivr.net/npm/world-atlas@1/world/110m.tsv",
);
let data = await response.text();
let df = pl.readCSV(data, { sep: "\t" });

df

scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,geou_dif,geounit,gu_a3,su_dif,subunit,su_a3,brk_diff,name,name_long,brk_a3,brk_name,brk_group,abbrev,postal,formal_en,formal_fr,note_adm0,note_brk,name_sort,name_alt,mapcolor7,mapcolor8,mapcolor9,mapcolor13,pop_est,gdp_md_est,pop_year,lastcensus,gdp_year,economy,income_grp,wikipedia,fips_10,iso_a2,iso_a3,iso_n3,un_a3,wb_a2,wb_a3,woe_id,adm0_a3_is,adm0_a3_us,adm0_a3_un,adm0_a3_wb,continent,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart
1,Admin-0 country,6,Kosovo,KOS,0,2,Sovereign country,Kosovo,KOS,0,Kosovo,KOS,0,Kosovo,KOS,1,Kosovo,Kosovo,B57,Kosovo,null,Kos.,KO,Republic of Kosovo,null,null,Self admin.; Claimed by Serbia,Kosovo,null,2,2,3,11,1804838,5352,-99,1981,-99,6. Developing region,4. Lower middle income,-99,null,-99,-99,-99,-99,KV,KSV,-99,SRB,KOS,-99,-99,Europe,Europe,Southern Europe,Europe & Central Asia,6,6,4,-99,1
1,Admin-0 country,5,Somaliland,SOL,0,2,Indeterminate,Somaliland,SOL,0,Somaliland,SOL,0,Somaliland,SOL,1,Somaliland,Somaliland,B30,Somaliland,null,Solnd.,SL,Republic of Somaliland,null,Self admin.,Self admin.; Claimed by Somalia,Somaliland,null,3,6,5,2,3500000,12250,-99,-99,-99,6. Developing region,4. Lower middle income,-99,null,-99,-99,-99,-99,-99,-99,-99,SOM,SOM,-99,-99,Africa,Africa,Eastern Africa,Sub-Saharan Africa,10,10,6,-99,1
1,Admin-0 country,6,Northern Cyprus,CYN,0,2,Sovereign country,Northern Cyprus,CYN,0,Northern Cyprus,CYN,0,Northern Cyprus,CYN,1,N. Cyprus,Northern Cyprus,B20,N. Cyprus,null,N. Cy.,CN,Turkish Republic of Northern Cyprus,null,Self admin.,Self admin.; Claimed by Cyprus,"Cyprus, Northern",null,3,1,4,8,265100,3600,-99,-99,-99,6. Developing region,3. Upper middle income,-99,null,-99,-99,-99,-99,-99,-99,-99,CYP,CYP,-99,-99,Asia,Asia,Western Asia,Europe & Central Asia,9,15,6,-99,1
1,Admin-0 country,3,Afghanistan,AFG,0,2,Sovereign country,Afghanistan,AFG,0,Afghanistan,AFG,0,Afghanistan,AFG,0,Afghanistan,Afghanistan,AFG,Afghanistan,null,Afg.,AF,Islamic State of Afghanistan,null,null,null,Afghanistan,null,5,6,8,7,28400000,22270,-99,1979,-99,7. Least developed region,5. Low income,-99,null,AF,AFG,4,4,AF,AFG,-99,AFG,AFG,-99,-99,Asia,Asia,Southern Asia,South Asia,11,11,4,-99,1
1,Admin-0 country,3,Angola,AGO,0,2,Sovereign country,Angola,AGO,0,Angola,AGO,0,Angola,AGO,0,Angola,Angola,AGO,Angola,null,Ang.,AO,People's Republic of Angola,null,null,null,Angola,null,3,2,6,1,12799293,110300,-99,1970,-99,7. Least developed region,3. Upper middle income,-99,null,AO,AGO,24,24,AO,AGO,-99,AGO,AGO,-99,-99,Africa,Africa,Middle Africa,Sub-Saharan Africa,6,6,4,-99,1
1,Admin-0 country,6,Albania,ALB,0,2,Sovereign country,Albania,ALB,0,Albania,ALB,0,Albania,ALB,0,Albania,Albania,ALB,Albania,null,Alb.,AL,Republic of Albania,null,null,null,Albania,null,1,4,1,6,3639453,21810,-99,2001,-99,6. Developing region,4. Lower middle income,-99,null,AL,ALB,8,8,AL,ALB,-99,ALB,ALB,-99,-99,Europe,Europe,Southern Europe,Europe & Central Asia,7,7,4,-99,1
1,Admin-0 country,4,United Arab Emirates,ARE,0,2,Sovereign country,United Arab Emirates,ARE,0,United Arab Emirates,ARE,0,United Arab Emirates,ARE,0,United Arab Emirates,United Arab Emirates,ARE,United Arab Emirates,null,U.A.E.,AE,United Arab Emirates,null,null,null,United Arab Emirates,null,2,1,3,3,4798491,184300,-99,2010,-99,6. Developing region,2. High income: nonOECD,-99,null,AE,ARE,784,784,AE,ARE,-99,ARE,ARE,-99,-99,Asia,Asia,Western Asia,Middle East & North Africa,20,20,6,-99,1
1,Admin-0 country,2,Argentina,ARG,0,2,Sovereign country,Argentina,ARG,0,Argentina,ARG,0,Argentina,ARG,0,Argentina,Argentina,ARG,Argentina,null,Arg.,AR,Argentine Republic,null,null,null,Argentina,null,3,1,3,13,40913584,573900,-99,2010,-99,5. Emerging region: G20,3. Upper middle income,-99,null,AR,ARG,32,32,AR,ARG,-99,ARG,ARG,-99,-99,South America,Americas,South America,Latin America & Caribbean,9,9,4,-99,1
1,Admin-0 country,6,Armenia,ARM,0,2,Sovereign country,Armenia,ARM,0,Armenia,ARM,0,Armenia,A